## Load and maxfilter example data 

In [5]:
import os
import pandas as pd
import numpy as np

import mne
from mne.datasets import sample

data_path = sample.data_path()
fname = data_path + '/MEG/sample/sample_audvis_raw.fif'
raw = mne.io.read_raw_fif(fname, preload=True)

fine_cal_file = os.path.join(data_path, 'SSS', 'sss_cal_mgh.dat')
crosstalk_file = os.path.join(data_path, 'SSS', 'ct_sparse_mgh.fif')
head_pos_file = os.path.join(mne.datasets.testing.data_path(), 'SSS', 'test_move_anon_raw.pos')
head_pos = mne.chpi.read_head_pos(head_pos_file)

raw_tsss_mc = mne.preprocessing.maxwell_filter(raw, cross_talk=crosstalk_file, calibration=fine_cal_file, 
                                            st_duration=10, head_pos=head_pos)

run1_tsss_mc = raw_tsss_mc.copy().crop(tmin=0, tmax=200)
run2_tsss_mc = raw_tsss_mc.copy().crop(tmin=200)

Opening raw data file /Users/valery/mne_data/MNE-sample-data/MEG/sample/sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Current compensation grade : 0
Reading 0 ... 166799  =      0.000 ...   277.714 secs...
Maxwell filtering raw data
    Using loaded raw data
    Bad MEG channels being reconstructed: ['MEG 2443']
    Processing 204 gradiometers and 102 magnetometers
    Using fine calibration sss_cal_mgh.dat
        Adjusting non-orthogonal EX and EY
        Adjusted coil positions by (μ ± σ): 0.5° ± 0.4° (max: 2.1°)
    Automatic origin fit: head of radius 91.0 mm
    Using origin -4.1, 16.0, 51.7 mm in the head frame
    Processing data using tSSS with st_duration=9.999750418577312
    Spatiotemporal window did not fit evenly intocontiguous data segment. 7.72 seconds were lumped into the previous window.
   

### Save files

In [ ]:
root_path = '/Users/valery/MEG/Example/'
case_name = 'case_1'
case_path = root_path+case_name+'/'

os.makedirs(case_path + 'MEG_data/tsss_mc/',exist_ok=True)

run1_tsss_mc.save(case_path + 'MEG_data/tsss_mc/case1_run1_tsss_mc.fif')
run2_tsss_mc.save(case_path + 'MEG_data/tsss_mc/case1_run2_tsss_mc.fif')

## Directories

In [1]:
import os
import pandas as pd
import numpy as np

import circus_templates
import meg_preprocessing
import run_circus

root_path = '/Users/valery/MEG/Example/'
case_name = 'case_1'
case_path = root_path+case_name+'/'

prep = meg_preprocessing.Preprocessing(dir_root=root_path, case=case_name, directory=case_path)


### Plot PSD

In [ ]:
prep.concatenate_files()
prep.plot_psd()
prep.update_cases_files(first_time=True)

### Open the file *all_cases_files.xlsx* in the root folder
Manually write **False** in the column *skipped* to select the file for the further analysis

In [9]:
prep.update_cases_files()
prep.ICA_plots([prep.file_fif_for_circus])
prep.update_cases_files()

ICA fitting, artefscts detecting, properties and sources plotting...


         Files  [█████████████████████████████████████████████] 1/1 [05:31<05:31, 331.32s/it]

/anaconda3/envs/mne/lib/python3.6/site-packages/sklearn/decomposition/fastica_.py:119: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)


Automatic bad components for case1_run1_tsss_mc.fif:  [46, 23]


### Visually select bad components and write their numbers in the column *Manual_components* in the file *all_cases_files.xlsx*
For example: 0, 1

In [10]:
prep.apply_ica()

File case1_run1_tsss_mc.fif saved as case1_run1_tsss_mc_0.npy in the folder /Users/valery/MEG/Example/case_1/Spyking_circus/case_1_case1_run1_tsss_mc


## Run Spyking Circus

In [2]:
prep.information_for_circus()
sc = run_circus.Circus(prep.dir_case, prep.case, prep.folder_for_circus, 
                      prep.file_fif_for_circus, prep.mspikes_for_circus, MAD=5.0)

sc.params_iterations(run_spc=True, ROC=False)



Folder: /Users/valery/MEG/Example/case_1/Spyking_circus/case_1_case1_run1_tsss_mc/
File: case1_run1_tsss_mc_0.npy
Number manual spikes: 0 
Parameters: N_t = 80, Cut off = 9, Threshold = 5.0, cc_merge = 0.9



       Sensors  [█████████████████████████████████████████████] 2/2 [04:25<01:39, 132.53s/it]

## Plot all templates

In [3]:
for sens in ['grad', 'mag']:
    temp = circus_templates.Templates(prep.dir_case, prep.case, prep.file_fif_for_circus, 
                               sc.sensors_params[sens], sensors=sens, n_sp=5)
    temp.plot_all_templates('{}{}'.format(prep.dir_tsss, prep.file_fif_for_circus), 'Mac')

     Templates  [█████████████████████████████████████████████] 7/7 [02:22<00:17, 20.33s/it]

     Templates  [█████████████████████████████████████████████] 1/1 [00:23<00:23, 22.67s/it]